In [6]:
# 04_multihorizon_baseline_ratio.ipynb

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.arima.model import ARIMA

sns.set_theme(font_scale=1.1)

DATA_FOLDER = '../data'

all_coins = [
    "Aave",
    "BinanceCoin",
    "Bitcoin",
    "Cardano",
    "ChainLink",
    "Cosmos",
    "CryptocomCoin",
    "Dogecoin",
    "EOS",
    "Ethereum",
    "Iota",
    "Litecoin",
    "Monero",
    "NEM",
    "Polkadot",
    "Solana",
    "Stellar",
    "Tether",
    "Tron",
    "Uniswap",
    "USDCoin",
    "WrappedBitcoin",
    "XRP"
]

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def find_best_arima(train_series, p_range=(0,2), d_range=(0,2), q_range=(0,2)):
    best_aic = float('inf')
    best_order = None
    best_model_fit = None

    for p in range(p_range[0], p_range[1]+1):
        for d in range(d_range[0], d_range[1]+1):
            for q in range(q_range[0], q_range[1]+1):
                try:
                    model = ARIMA(train_series, order=(p,d,q))
                    fitted = model.fit()
                    if fitted.aic < best_aic:
                        best_aic = fitted.aic
                        best_order = (p,d,q)
                        best_model_fit = fitted
                except:
                    continue
    return best_order, best_model_fit

# We'll look for these horizon columns
horizons = ['Close_t+1','Close_t+7','Close_t+30','Close_t+90']

results = []

for coin in all_coins:
    print(f"\n=== Multi-Horizon Baseline for {coin} ===")
    train_csv = f"{coin.lower()}_train_multi.csv"
    test_csv  = f"{coin.lower()}_test_multi.csv"

    train_path = os.path.join(DATA_FOLDER, train_csv)
    test_path  = os.path.join(DATA_FOLDER, test_csv)

    if not (os.path.exists(train_path) and os.path.exists(test_path)):
        print(f"[{coin}] Missing multi-horizon train/test, skipping coin.")
        continue

    train_df = pd.read_csv(train_path, parse_dates=['Date'])
    test_df  = pd.read_csv(test_path,  parse_dates=['Date'])

    if len(train_df) < 20:
        print(f"[{coin}] Train set <20 rows, skipping ARIMA.")
        continue
    if len(test_df) == 0:
        print(f"[{coin}] Test set empty, skipping coin.")
        continue

    # ARIMA on train's 'Close'
    train_close = train_df['Close']
    best_order, best_fit = find_best_arima(train_close)
    if best_fit is None:
        print(f"[{coin}] No ARIMA found for p,d,q up to (2,2,2). Skipping coin.")
        continue

    # We'll forecast the entire test length
    steps_needed = len(test_df)
    if steps_needed <= 0:
        print(f"[{coin}] steps_needed={steps_needed}, skipping coin.")
        continue

    try:
        forecast_values = best_fit.forecast(steps=steps_needed)
        test_df['arima_pred_close'] = forecast_values.values
    except ValueError as e:
        print(f"[{coin}] ARIMA forecast error: {e}")
        continue

    # Evaluate each horizon
    for horizon_col in horizons:
        if horizon_col not in test_df.columns:
            continue

        # If it's all NaN, skip
        if test_df[horizon_col].isna().all():
            print(f"[{coin}] {horizon_col} is all NaN, skipping horizon.")
            continue

        # Naive approach
        test_df[f'naive_{horizon_col}'] = test_df['Close']  # same-day price
        y_true = test_df[horizon_col]
        y_naive = test_df[f'naive_{horizon_col}']

        naive_rmse_val = rmse(y_true, y_naive)
        naive_mae_val  = mae(y_true, y_naive)

        # ARIMA approach
        y_arima = test_df['arima_pred_close']
        arima_rmse_val = rmse(y_true, y_arima)
        arima_mae_val  = mae(y_true, y_arima)

        print(f"{coin} - {horizon_col}: NaiveRMSE={naive_rmse_val:.2f}, ARIMARMSE={arima_rmse_val:.2f}")

        results.append({
            'Coin': coin,
            'Horizon': horizon_col,
            'Naive_RMSE': naive_rmse_val,
            'Naive_MAE': naive_mae_val,
            'ARIMA_Order': best_order,
            'ARIMA_RMSE': arima_rmse_val,
            'ARIMA_MAE': arima_mae_val
        })

# Save results
results_df = pd.DataFrame(results)
out_csv = os.path.join(DATA_FOLDER, 'baseline_results_multihorizon.csv')
results_df.to_csv(out_csv, index=False)
print(f"\n=== Saved multi-horizon baseline results to: {out_csv}")
print(results_df.head(30))



=== Multi-Horizon Baseline for Aave ===


c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('N

Aave - Close_t+1: NaiveRMSE=18.27, ARIMARMSE=31.15
Aave - Close_t+7: NaiveRMSE=37.91, ARIMARMSE=26.02
Aave - Close_t+30: NaiveRMSE=63.20, ARIMARMSE=70.79
Aave - Close_t+90: NaiveRMSE=110.12, ARIMARMSE=101.00

=== Multi-Horizon Baseline for BinanceCoin ===


c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('N

BinanceCoin - Close_t+1: NaiveRMSE=13.63, ARIMARMSE=115.62
BinanceCoin - Close_t+7: NaiveRMSE=40.09, ARIMARMSE=138.55
BinanceCoin - Close_t+30: NaiveRMSE=112.77, ARIMARMSE=213.25
BinanceCoin - Close_t+90: NaiveRMSE=227.23, ARIMARMSE=278.97

=== Multi-Horizon Baseline for Bitcoin ===


...

c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


WrappedBitcoin - Close_t+1: NaiveRMSE=1667.78, ARIMARMSE=26119.09
WrappedBitcoin - Close_t+7: NaiveRMSE=4788.57, ARIMARMSE=27637.09
WrappedBitcoin - Close_t+30: NaiveRMSE=10129.64, ARIMARMSE=31865.84
WrappedBitcoin - Close_t+90: NaiveRMSE=24861.82, ARIMARMSE=34435.13

=== Multi-Horizon Baseline for XRP ===


c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary 

XRP - Close_t+1: NaiveRMSE=0.03, ARIMARMSE=0.17
XRP - Close_t+7: NaiveRMSE=0.10, ARIMARMSE=0.22
XRP - Close_t+30: NaiveRMSE=0.24, ARIMARMSE=0.33
XRP - Close_t+90: NaiveRMSE=0.34, ARIMARMSE=0.43

=== Saved multi-horizon baseline results to: ../data\baseline_results_multihorizon.csv
             Coin     Horizon    Naive_RMSE    Naive_MAE ARIMA_Order  \
0            Aave   Close_t+1     18.265138    15.094809   (1, 1, 2)   
1            Aave   Close_t+7     37.906837    32.636441   (1, 1, 2)   
2            Aave  Close_t+30     63.201110    52.020578   (1, 1, 2)   
3            Aave  Close_t+90    110.123263   101.843564   (1, 1, 2)   
4     BinanceCoin   Close_t+1     13.630565     5.007229   (2, 1, 2)   
5     BinanceCoin   Close_t+7     40.087401    16.957944   (2, 1, 2)   
6     BinanceCoin  Close_t+30    112.771262    61.253317   (2, 1, 2)   
7     BinanceCoin  Close_t+90    227.229684   143.219464   (2, 1, 2)   
8         Bitcoin   Close_t+1    902.244539   452.576487   (2, 1, 2)  

c:\Users\lukeb\crypto-forecasting-capstone\venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
